# K-Means Notebook

<font color='royalblue'>This notebook is meant to test our K-Means clustering algorithm implemented in the python script `KMeans.py`. </font><br><br>

<font color='royalblue'> To test our model we will use the <a href="https://www.kaggle.com/datasets/fernandol/countries-of-the-world"> World Countries Dataset </a> collected by the US Government in 2013. We'll keep in mind that the data we use is not that recent. Our goal in this notebook will be to use our KMeans class to group countries. We expect the countries with equivalent levels of development to be in the same cluster. The United Nations publish each year a <a href="https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/WESP2020_Annex.pdf">global countries classification</a> where every state of planet is assigned to one of the following groups : developed economies, devoloping economies and economies in transition. </font><br>

<font color='royalblue'> Let's see if when we run our KMeans clustering on the world countries data with K=3 clusters if the groups created are representative of the levels of development stated by the UN or not so much. </font>

<font color='gray'><br>The only libraries you need are `numpy` and `pandas`. Then you should be up and running to execute the whole notebook 👍 </font>

## Imports

In [1]:
import scipy
import KMeans
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from KMeans import *
from scipy import stats

In [2]:
#dataset
dataset_path = "../datasets/world_countries.csv"

df = pd.read_csv(dataset_path, index_col=False, decimal=',')

## Dataset description

In [3]:
df.describe()

,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
count,2.270000e+02,2.270000e+02,227.000000,227.000000,224.000000,224.000000,226.000000,209.000000,223.000000,225.000000,225.000000,225.000000,205.000000,224.000000,223.000000,212.000000,211.000000,212.000000
mean,2.874028e+07,5.982270e+05,379.047137,21.165330,0.038125,35.506964,9689.823009,82.838278,236.061435,13.797111,4.564222,81.638311,2.139024,22.114732,9.241345,0.150844,0.282711,0.565283
std,1.178913e+08,1.790282e+06,1660.185825,72.286863,4.889269,35.389899,10049.138513,19.722173,227.991829,13.040402,8.361470,16.140835,0.699397,11.176716,4.990026,0.146798,0.138272,0.165841
min,7.026000e+03,2.000000e+00,0.000000,0.000000,-20.990000,2.290000,500.000000,17.600000,0.200000,0.000000,0.000000,33.330000,1.000000,7.290000,2.290000,0.000000,0.020000,0.062000
25%,4.376240e+05,4.647500e+03,29.150000,0.100000,-0.927500,8.150000,1900.000000,70.600000,37.800000,3.220000,0.190000,71.650000,2.000000,12.672500,5.910000,0.037750,0.193000,0.429250
50%,4.786994e+06,8.660000e+04,78.800000,0.730000,0.000000,21.000000,5550.000000,92.500000,176.200000,10.420000,1.030000,85.700000,2.000000,18.790000,7.840000,0.099000,0.272000,0.571000
75%,1.749777e+07,4.418110e+05,190.150000,10.345000,0.997500,55.705000,15700.000000,98.000000,389.650000,20.000000,4.440000,95.440000,3.000000,29.820000,10.605000,0.221000,0.341000,0.678500
max,1.313974e+09,1.707520e+07,16271.500000,870.660000,23.060000,191.190000,55100.000000,100.000000,1035.600000,62.110000,50.680000,100.000000,4.000000,50.730000,29.740000,0.769000,0.906000,0.954000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Country                             227 non-null    object 
 1   Region                              227 non-null    object 
 2   Population                          227 non-null    int64  
 3   Area (sq. mi.)                      227 non-null    int64  
 4   Pop. Density (per sq. mi.)          227 non-null    float64
 5   Coastline (coast/area ratio)        227 non-null    float64
 6   Net migration                       224 non-null    float64
 7   Infant mortality (per 1000 births)  224 non-null    float64
 8   GDP ($ per capita)                  226 non-null    float64
 9   Literacy (%)                        209 non-null    float64
 10  Phones (per 1000)                   223 non-null    float64
 11  Arable (%)                          225 non-n

## Dataset cleaning

<font color='royalblue'> K-Means is an algorithm very sensitive to outliers. So before fitting the model to our data we will (i) delete countries for which NaN values appear (ii) remove outliers ie. countries which features distribution is very far from other countries.</font><br><br>

In [5]:
# features selection
df = df[['Country', 'Population', 'Net migration', 'Infant mortality (per 1000 births)',
         'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)', 'Birthrate', 'Deathrate',
         'Coastline (coast/area ratio)', 'Agriculture', 'Industry', 'Service']]

In [6]:
df = df.dropna()

In [7]:
outliers = df[(np.abs(scipy.stats.zscore(df.iloc[:,1:])) >= 3).all(axis=1)]

df = df[(np.abs(scipy.stats.zscore(df.iloc[:,1:])) < 3).all(axis=1)]

print(df)

                Country  Population  Net migration  \
1              Albania      3581655          -4.93   
2              Algeria     32930091          -0.39   
6             Anguilla        13477          10.76   
7    Antigua & Barbuda        69108          -6.15   
8            Argentina     39921833           0.61   
..                  ...         ...            ...   
218          Venezuela     25730435          -0.04   
219            Vietnam     84402966          -0.45   
224              Yemen     21456188           0.00   
225             Zambia     11502010           0.00   
226           Zimbabwe     12236805           0.00   

     Infant mortality (per 1000 births)  GDP ($ per capita)  Literacy (%)  \
1                                 21.52              4500.0          86.5   
2                                 31.00              6000.0          70.0   
6                                 21.03              8600.0          95.0   
7                                 19.46    

In [8]:
dataset = np.array(df)

X_train = dataset[:,1:]

## Countries clustering

In [9]:
model = KMeans()

In [10]:
clusters = model.fit(X_train, K=3, n_epochs_max = 50)

End of clustering
	7 epochs done 
	0 changes on last epochs 


In [11]:
#countries = np.reshape(dataset[:,0], (dataset.shape[0],1))
category = np.reshape(clusters[:,-1], (clusters.shape[0],1))

dataset_ = np.hstack([dataset, category])

for c in np.unique(category):
    
    print(f"CATEGORY {int(c)}\n")
    print(dataset_[dataset_[:,-1] == c, 0])
    print(f"\n------------------------------\n")



CATEGORY 0

['Bangladesh ' 'Benin ' 'Bhutan ' 'Burkina Faso ' 'Burma ' 'Burundi '
 'Cambodia ' 'Cameroon ' 'Central African Rep. ' 'Chad ' 'Comoros '
 'Congo, Dem. Rep. ' 'Congo, Repub. of the ' "Cote d'Ivoire " 'Djibouti '
 'Eritrea ' 'Ethiopia ' 'Gabon ' 'Gambia, The ' 'Ghana ' 'Guinea '
 'Haiti ' 'Kenya ' 'Laos ' 'Madagascar ' 'Malawi ' 'Mali ' 'Mauritania '
 'Mozambique ' 'Nepal ' 'Nigeria ' 'Pakistan ' 'Papua New Guinea '
 'Rwanda ' 'Senegal ' 'Sierra Leone ' 'Sudan ' 'Tajikistan ' 'Tanzania '
 'Togo ' 'Uganda ' 'Uzbekistan ' 'Vanuatu ' 'Yemen ' 'Zambia ' 'Zimbabwe ']

------------------------------

CATEGORY 1

['Albania ' 'Algeria ' 'Argentina ' 'Armenia ' 'Azerbaijan ' 'Belarus '
 'Belize ' 'Bolivia ' 'Brazil ' 'Brunei ' 'Bulgaria ' 'Cape Verde '
 'Chile ' 'Colombia ' 'Costa Rica ' 'Cuba ' 'Dominica '
 'Dominican Republic ' 'Ecuador ' 'Egypt ' 'El Salvador ' 'Fiji '
 'Georgia ' 'Grenada ' 'Guatemala ' 'Guyana ' 'Honduras ' 'Indonesia '
 'Iran ' 'Iraq ' 'Jamaica ' 'Jordan ' 'Kaz